# Neural Net Estimation Code for Surrogate Model for Sensitivity Analysis for Macroeconomics of Aging 
To run this notebook, use the `dynviz-dev` virtual conda environment, which can be built using the `environment.yml` file located in the same directory as this notebook (`conda env create -f environment.yml`).

This notebook builds a surrogate model for a sensitivity analysis tool for the macroeconomic impacts of improvements in biological aging. In this project, each simulation of a reform scenario of the OG-USA macroeconomic model takes about 30 minutes on a fairly powerful machine. From our simulations, we wanted to test the sensitivity of the results to six variables. If we wanted to look at all the permutations of the model for three values in each of the six input variables, that would equal 729 runs of the model.

If we assume that the equilibrium solutions are continuous and smooth between those points, we could fit a surrogate model function to those six input variables and an output variable (a regression model).

The following $X$ and $y$ data represent $N=27,472$ observations from a linear interpolant of 472 simulations of our model, each with different values of the $X$ input variables. These original 472 simulations represent all the permutations of the max and min values of the 6 input variables in $X$ ($N\times 6$ matrix) plus some remaining interior point observations. (The last 472 observations are the original 472 observations.)

The final caveat is that we need our final interpolant to be a tensorflow neural net because that is the only model that the `tensorflow.js` library can host on a web page.

**THE PROBLEM WE ARE STUCK ON (TRYING TO SOLVE)**

We assumed that a deep net would fit the data well and provide a smooth interpolant of our training data and test data. However, as can be seen below, our neural net does not fit the training data very well. We wanted to know if you have any insights into how to fit a neural net to these data better.

In [52]:
# imports
import numpy as np
import os
import pandas as pd
import scipy.interpolate as si
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split

## 1. Prep the data
The following data contains 1 predicted variable $y$ which is an $N\times 1$ vector and one matrix of 6 input variables $X$ which is an $N\times 6$ matrix, where $N=27,472$. 

In [53]:
# Set the current directory as the cur_dir path
cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, 'data')

# Read in the data
X = np.loadtxt(os.path.join(data_dir, 'X.txt'), delimiter=',')
y = np.loadtxt(os.path.join(data_dir, 'y.txt'), delimiter=',').reshape(-1, 1)
print(f"The shape of X is {X.shape}")
print(f"The shape of y is {y.shape}")

The shape of X is (27472, 6)
The shape of y is (27472, 1)


In [54]:
# train-test split for model evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, shuffle=True
)

Here is the linear N-dimensional interpolant that we fit to our sparse runs of 472 data points in this $(X,y)$ space. This interpolant `interpolant_avg_diff()` is an estimated `scipy.interpolate.LinearNDInterpolator` object. This would be useful, e.g., if you want to use the surrogate model to generate new data points for each epoch.

In [55]:
interpolant_avg_diff = pickle.load(
    open(
        os.path.join(data_dir, "interpolant_avg_diff.pkl"), "rb"
    )
)
# using
prediction = interpolant_avg_diff(X_test)
print("Predictions = ", prediction[:5])

Predictions =  [[1155.22108399]
 [1793.49360906]
 [1269.150218  ]
 [ 830.42326458]
 [ 472.91757689]]


## 2. Set up the NN model

In [ ]:
# set some hyperparameters
nodes_per_layer = 20 # multiplied by num inputs
activation_function = "relu"
num_epochs = 200
learning_rate = 0.001
# set up early stopping
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5000)

In [57]:
# Layer setting
num_input = X.shape[1]
num_hidden_layers = 7
num_hidden = np.zeros(num_hidden_layers)
# Start with more nodes per layer and decrease
for i in range(num_hidden_layers):
    num_hidden[i] = (nodes_per_layer - i * 2) * num_input
# num_hidden = nodes_per_layer * num_input
num_output = y.shape[1]
layers_dim = [num_input, num_hidden, num_output]
print("Dimensions of each layer are {}".format(layers_dim))

Dimensions of each layer are [6, array([120., 108.,  96.,  84.,  72.,  60.,  48.]), 1]


In [ ]:
# Define the normalization function of the output
def normalize_output(data):
    data_max = y_train.max()
    data_min = y_train.min()
    return (data - data_min) / (data_max - data_min)

In [59]:
# Create a normalization layer for the inputs
norm_layer = tf.keras.layers.Normalization(input_shape=[num_input,], axis=-1)
# norm_layer = tf.keras.layers.Normalization(input_shape=[num_input,], axis=None)

# Initialize the weights
initializer = tf.keras.initializers.HeNormal(99)
# initializer = tf.keras.initializers.HeUniform()

# Adapt the layer to your training data
norm_layer.adapt(X_train)
nn = tf.keras.Sequential(
    [
        norm_layer,
        keras.layers.Dense(num_hidden[0], activation=activation_function, kernel_initializer=initializer),
        keras.layers.Dense(num_hidden[1], activation=activation_function, kernel_initializer=initializer),
        keras.layers.Dense(num_hidden[2], activation=activation_function, kernel_initializer=initializer),
        keras.layers.Dense(num_hidden[3], activation=activation_function, kernel_initializer=initializer),
        keras.layers.Dense(num_hidden[4], activation=activation_function, kernel_initializer=initializer),
        keras.layers.Dense(num_hidden[5], activation=activation_function, kernel_initializer=initializer),
        keras.layers.Dense(num_hidden[6], activation=activation_function, kernel_initializer=initializer),
        keras.layers.Dense(num_output, kernel_initializer=initializer),
    ]
)
print(nn.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_4 (Normaliza  (None, 6)                 13        
 tion)                                                           
                                                                 
 dense_32 (Dense)            (None, 120)               840       
                                                                 
 dense_33 (Dense)            (None, 108)               13068     
                                                                 
 dense_34 (Dense)            (None, 96)                10464     
                                                                 
 dense_35 (Dense)            (None, 84)                8148      
                                                                 
 dense_36 (Dense)            (None, 72)                6120      
                                                      

In [60]:
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)
nn.compile(optimizer=optimizer, loss=loss_fn, metrics=['mean_squared_error'])

## 3. Train the NN model

In [ ]:
history = nn.fit(
    X_train, normalize_output(y_train),
    epochs=num_epochs,
    callbacks=[callback],
    # sample_weight=sample_weights,
    batch_size=32,
    validation_data=(X_test,  normalize_output(y_test)),
    validation_freq=5,   # Evaluate every 5 epochs
    verbose=1
)

Epoch 1/10000
687/687 [==============================] - 1s 1ms/step - loss: 0.0040 - mean_squared_error: 0.0040
Epoch 2/10000
687/687 [==============================] - 1s 1ms/step - loss: 3.7938e-04 - mean_squared_error: 3.7938e-04
Epoch 3/10000
687/687 [==============================] - 1s 1ms/step - loss: 2.5159e-04 - mean_squared_error: 2.5159e-04
Epoch 4/10000
687/687 [==============================] - 1s 1ms/step - loss: 1.9259e-04 - mean_squared_error: 1.9259e-04
Epoch 5/10000
687/687 [==============================] - 1s 1ms/step - loss: 1.6719e-04 - mean_squared_error: 1.6719e-04 - val_loss: 1.8621e-04 - val_mean_squared_error: 1.8621e-04
Epoch 6/10000
687/687 [==============================] - 1s 1ms/step - loss: 1.4442e-04 - mean_squared_error: 1.4442e-04
Epoch 7/10000
687/687 [==============================] - 1s 1ms/step - loss: 1.5040e-04 - mean_squared_error: 1.5040e-04
Epoch 8/10000
687/687 [==============================] - 1s 1ms/step - loss: 1.4403e-04 - mean_square

In [ ]:
# Plot training and validation loss
plt.plot(np.arange(num_epochs), history.history['loss'], label='Train Loss')
plt.plot(
    np.linspace(0, num_epochs, num=int(np.ceil(num_epochs/5))),
    history.history['val_loss'], label='Test Loss', marker='o'
)
plt.ylim(0, 0.0005)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train vs Test Loss')
plt.legend()
plt.show()

## 4. Check the fit of the model

In [ ]:
nn.evaluate(X,  normalize_output(y), verbose=2)

### 4.1. Checking against the original data

In [ ]:
output_variable = "avg_diff"
predictions_normed = nn.predict(X)
# Rescale the predictions
predictions = predictions_normed * (y_train.max() - y_train.min()) + y_train.min()
y_pred = predictions
y_diff = y - y_pred
print(
    f"The mean abs diff between y and y_pred is {np.absolute(y_diff).mean()}."
)
print(f"The max diff between y and y_pred is {y_diff.max()}.")
print(f"The min diff between y and y_pred is {y_diff.min()}.")

In [ ]:
normalization_layer = nn.get_layer('normalization')
mean = normalization_layer.mean.numpy()
variance = normalization_layer.variance.numpy()
print("Mean: ", mean)
print("Std dev: ", variance**(1/2))

### 4.2. Check closeness to runs used in Silver Linings Book

There are a number of runs included in the Silver Linings book that we want to hit **extremely** closely. These happen to be the last 472 observations of the $X$ and $y$ data objects. We can check how close the surrogate model is to these runs.

In [ ]:
# Load model
X_book = X[-472:, :]
y_book = y[-472:, :]

# predict using the model
y_book_pred_norm = nn.predict(X_book)
y_book_pred = y_book_pred_norm * (y_train.max() - y_train.min()) + y_train.min()
y_book_diff = y_book - y_book_pred
print(
    f"The mean abs diff between y_book_pred and y_book " +
    f"is {np.absolute(y_book_diff).mean()}."
)
print(f"The max diff between y_book_pred and y_book is {y_book_diff.max()}.")
print(f"The min diff between y_book_pred and y_book is {y_book_diff.min()}.")
